In [1]:
from english_words import english_words_set
import pandas as pd
import numpy as np

In [2]:
# make a list of 5 lenght words without non-alphas, and no proper nouns
words5 = []

for word in english_words_set:
    if len(word) == 5 and word[0].islower() and word.isalpha():
        words5.append(word)

In [3]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [4]:
df_words = pd.DataFrame(words5)
df_words.columns=['words']

In [5]:
df_words['first'] = df_words['words'].str.slice(0,1)
df_words['second'] = df_words['words'].str.slice(1,2)
df_words['third'] = df_words['words'].str.slice(2,3)
df_words['fourth'] = df_words['words'].str.slice(3,4)
df_words['fifth'] = df_words['words'].str.slice(4,5)

In [6]:
df_words.head()

,words,first,second,third,fourth,fifth
0,sugar,s,u,g,a,r
1,mural,m,u,r,a,l
2,flake,f,l,a,k,e
3,clout,c,l,o,u,t
4,shrew,s,h,r,e,w


In [7]:
aplhas = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [8]:
dfalpha = pd.DataFrame(aplhas)
dfalpha.columns = ['letters']
dfalpha.set_index('letters', inplace=True)

In [9]:
# build the frequerncy table from each smaller dataframe
df_counts = df_words['first'].value_counts()
df_counts_second = df_words['second'].value_counts()
df_counts_third = df_words['third'].value_counts()
df_counts_fourth = df_words['fourth'].value_counts()
df_counts_fifth = df_words['fifth'].value_counts()

dfranks = pd.concat([dfalpha, df_counts], axis=1)
dfranks = pd.concat([dfranks, df_counts_second], axis=1)
dfranks = pd.concat([dfranks, df_counts_third], axis=1)
dfranks = pd.concat([dfranks, df_counts_fourth], axis=1)
dfranks = pd.concat([dfranks, df_counts_fifth], axis=1)

In [10]:
dfranks.head()

,first,second,third,fourth,fifth
a,157,334,334,199.0,93.0
b,203,18,56,25.0,10.0
c,224,42,61,162.0,41.0
d,107,19,78,74.0,110.0
e,72,262,197,261.0,468.0


In [11]:
dfranks['first']['a']

157

In [12]:
def freq_ranking(word):
    val = dfranks['first'][word[0]] + dfranks['second'][word[1]] +dfranks['third'][word[2]] + dfranks['fourth'][word[3]] +dfranks['fifth'][word[4]]
    return val

In [13]:
df_word_ranks = pd.DataFrame({'word' : [],'value':[]})

In [14]:
word_freq = {'word':'value'}
for word in words5:
    df_word_ranks.loc[len(df_word_ranks.index), 'word'] = word
    df_word_ranks.loc[len(df_word_ranks.index)-1, 'value'] = freq_ranking(word)

In [15]:
df_word_ranks.head()

,word,value
0,sugar,1016.0
1,mural,897.0
2,flake,1208.0
3,clout,1045.0
4,shrew,1029.0


In [16]:
df_word_ranks.to_csv('word_ranks')

In [17]:
print (df_word_ranks['word'][df_word_ranks['value'].idxmax()])
print (df_word_ranks['value'][df_word_ranks['value'].idxmax()])

slate
1569.0


In [160]:
# here we determine the next best word based on results of the best word per wordle feedback
# how to determine which items were elimiated
#  ~ before a contains statement makes it a "not" I was lucky to find this
display(df_word_ranks.loc[(~df_word_ranks['word'].str.contains('t'))])

,word,value
0,llama,813.0
1,chaos,924.0
2,monad,895.0
3,madam,789.0
4,annoy,902.0
...,...,...
2463,marsh,978.0
2464,franc,963.0
2465,swank,1086.0
2466,opium,502.0


In [19]:
grey = ['s','l','a','t','e']

In [20]:
df_sub = df_word_ranks
for l in grey:
    print(l)
    print(str(len(df_sub.index)))
    df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
df_sub

s
2470
l
1780
a
1240
t
738
e
495


,word,value
5,fjord,680.0
7,crumb,757.0
10,moody,1133.0
40,druid,847.0
63,bumpy,906.0
...,...,...
2378,frown,821.0
2398,cocky,1030.0
2400,mimic,612.0
2405,juicy,1029.0


In [21]:
print (df_sub['word'][df_sub['value'].idxmax()])
print (df_sub['value'][df_sub['value'].idxmax()])

crony
1308.0


In [31]:
def remove_notcontains(grey,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
    return df_sub

In [32]:
def remove_by_contains(letters,df):
    df_sub = df
    for l in letters:
        df_sub = (df_sub.loc[(df_sub['word'].str.contains(l))])
    return df_sub

In [79]:
def remove_by_location(loc,letter,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(df_sub['word'].str[loc]== letter)])
    return df_sub

In [90]:
grey = input()
df_options = df_word_ranks

In [48]:
df_options = remove_notcontains(grey,df_options)
print(df_options.head())
next_best = (df_options['word'][df_options['value'].idxmax()])
print(next_best)

      word  value
7    crumb  757.0
88   boric  922.0
101  runic  669.0
256  micro  640.0
275  choir  959.0
crony


In [67]:
confirmed = input()

In [ ]:
df_options = remove_by_contains(confirmed,df_options)
print(df_options.head())
next_best = (df_options['word'][df_options['value'].idxmax()])
print(next_best)

In [93]:
loc_letters = input('enter letter and locations in pairs [r3t4]')
print(loc_letters)

r1c4


In [94]:
for i in range(0,int(len(loc_letters)),2):
    print(str(i))
    print(loc_letters[i])
    print(loc_letters[i+1])
    df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
    print(df_options.head())
    next_best = (df_options['word'][df_options['value'].idxmax()])
    print(next_best)

0
r
1
     word  value
7   crumb  757.0
40  druid  847.0
66  order  785.0
78  droll  958.0
99  wrest  993.0
crane
2
c
4
       word  value
1711  franc  963.0
franc


In [96]:
# make the testing loop
# starting DF df_word_ranks

play =''
df_options = df_word_ranks
while play != 'no':
    round = 1
    while round < 6:
        round +=1 
        if round == 1:
            word = input('enter your guess:')
            pass

        # try:
        if round == 2:
            not_char = input('enter eliminated letters:[l,e,t,t,e,r]')
            is_char = input('enter confirmed letters:[y,e,s]')
            confirmed_locs = input('enter confirmed locations: [letter,location]')

            # eliminate options, and show the reduced best 5 choices
            if len(not_char)>1:
                df_options = remove_notcontains(not_char,df_word_ranks)
                print(df_options.head())
            if len(is_char)>0:
                df_options = remove_by_contains(is_char,df_options)
                print(df_options.head())
            
            if len(confirmed_locs) >0:
                # remove by letters in correct location
                for i in range(0,int(len(loc_letters)),2):
                    # print(str(i))
                    # print(loc_letters[i])
                    # print(loc_letters[i+1])
                    df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
                print(df_options.head())

        next_best = (df_options['word'][df_options['value'].idxmax()])
        print(next_best)
    # except:
    #     print(word)

     word   value
5   fjord   680.0
7   crumb   757.0
10  moody  1133.0
40  druid   847.0
63  bumpy   906.0
crony
    word   value
0  sugar  1016.0
1  mural   897.0
2  flake  1208.0
4  shrew  1029.0
6  lease  1340.0
      word   value
7    crumb   757.0
89   curia   896.0
108  wreck   825.0
119  craft  1140.0
181  cramp   966.0
Empty DataFrame
Columns: [word, value]
Index: []


ValueError: attempt to get argmax of an empty sequence